# Filtro Digital IIR - Pasa Bajos

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
import matplotlib
import scipy.signal as signal
import warnings
import mpld3
mpld3.enable_notebook()
warnings.filterwarnings("ignore")

## Método de Invarianza al Impulso

## Sistema analogico

In [3]:
#sistema
c = 2*2*np.pi
lambd = -2*2*np.pi
t_c = np.arange(0,10,0.01)
w = np.arange(0.001,500*2*np.pi,0.01)

sys_1 = signal.lti([0,c], [1,-lambd])
h_c = c*np.exp(lambd*t_c)

fig1, ax1 = plt.subplots()
# Muestras de las senial
ax1.plot(t_c, h_c)
# add labels
ax1.set_xlabel("Muestras", fontsize=16)
ax1.set_ylabel("Amplitud", fontsize=16)
ax1.grid(True)
ax1.legend(['Respuesta al Impulso Continua'])

# Respuesta en Frecuencia
w_log, mag_dB, fase = signal.bode(sys_1,w=w)
w, H_analog = signal.freqresp(sys_1,w=w)
mag_lin_analog = abs(H_analog) #magnitud
fase_analog = np.angle(H_analog, deg=True) #/np.pi #fase[rad]

fig2, ax2 = plt.subplots(1, 3, figsize=(16, 5))
#2 magnitud bode
ax2[0].semilogx(w_log/(2*np.pi),mag_dB)
ax2[0].set_title("Diagrama de Bode: Magnitud")
ax2[0].set_xlabel("Frecuencia [Hz]")  
ax2[0].set_ylabel("Magnitud [dB]")
ax2[0].grid(linestyle='--', which="both")#grilla punteada para escala logaritmica

#2 magnitud lineal
ax2[1].plot(w/(2*np.pi),mag_lin_analog)
ax2[1].set_title("Respuesta en frec.: Magnitud")
ax2[1].set_xlabel("Frecuencia [Hz]")  
ax2[1].set_ylabel("Magnitud [lineal]")
ax2[1].grid(linestyle='--', which="both")#grilla punteada para escala logaritmica

#3 fase
ax2[2].semilogx(w_log,fase)
ax2[2].set_title("Fase de la respuesta en frecuencia")
ax2[2].set_xlabel("Frecuencia [Hz]")
ax2[2].set_ylabel("Angulo[deg]")
ax2[2].grid(linestyle='--', which="both")#grilla punteada
fig2.show()

In [4]:
# Frecuencia de corte
print(c/(2*np.pi))

2.0


In [7]:
# Elegimos el tiempo de muestreo para discretizar la h(t)
f_max = 500
print(f_max)
T = 1/(2*f_max)# tiempo de muestreo
print(T)

500
0.001


## Metodo de Invarianza al Impulso

### Discretizamos la respuesta al impulso del Sistema analogico

In [8]:
n = np.arange(0,300,1)
h_d = T*c*np.exp(lambd*T*n) - T*c/2*signal.unit_impulse(n.shape[0]) 

fig3, ax4 = plt.subplots()
# Muestras de las senial
ax4.stem(n, h_d,use_line_collection=True)
# add labels
ax4.set_xlabel("Muestras", fontsize=16)
ax4.set_ylabel("Amplitud", fontsize=16)
ax4.grid(True)
ax4.legend(['Respuesta al Impulso Continua'])

### Sistema Digital equivalente

In [9]:
# Utilizamos la h[n] obtenida para implementar el sistema digital
sys_dig_1 = signal.dlti((T*c/2)*np.array([1,np.exp(lambd*T)]), [1,(-1)*np.exp(lambd*T)])

Omega = np.arange(1/1000,np.pi,0.001)

#respuesta en frecuencia
Omega_log, mag2_dB, fase2 = signal.dbode(sys_dig_1, w=Omega)
Omega2, H_dig = signal.dfreqresp(sys_dig_1,w=Omega)
mag_lin_dig = abs(H_dig) #magnitud
fase_dig = np.angle(H_dig, deg = True) #fase[deg]

fig4, ax4 = plt.subplots(1, 3, figsize=(16, 5))
#2 magnitud
ax4[0].plot(Omega/np.pi,mag2_dB)
ax4[0].set_title("Resp. en frecuencia: Magnitud")
ax4[0].set_xlabel("Frecuencia [rad/s]")  
ax4[0].set_ylabel("Magnitud [dB]")
ax4[0].xaxis.set_major_formatter(tck.FormatStrFormatter('%g $\pi$'))
ax4[0].grid(linestyle='--', which="both")#grilla punteada para escala logaritmica

ax4[1].plot(Omega2/np.pi,mag_lin_dig)
ax4[1].set_title("Resp. en frecuencia: Magnitud")
ax4[1].set_xlabel("Frecuencia [rad/s]")  
ax4[1].set_ylabel("Magnitud [lineal]")
ax4[1].xaxis.set_major_formatter(tck.FormatStrFormatter('%g $\pi$'))
ax4[1].grid(linestyle='--', which="both")#grilla punteada para escala logaritmica

#3 fase
ax4[2].plot(Omega/np.pi,fase2)
ax4[2].set_title("Fase de la respuesta en frecuencia")
ax4[2].set_xlabel("Frecuencia[rad/s]")
ax4[2].set_ylabel("Angulo[deg]")
ax4[2].grid(linestyle='--', which="both")#grilla punteada
ax4[2].xaxis.set_major_formatter(tck.FormatStrFormatter('%g $\pi$'))
#ax3.xaxis.set_major_locator(tck.MultipleLocator(base=0.5))
fig4.show()

### Superponemos ambas respuestas en frecuencia

In [10]:
#respuesta en frecuencia
f_dig = Omega2/np.pi*1/(2*T)
f_analog = w/(2*np.pi) 

fig5, ax5 = plt.subplots(1, 3, figsize=(16, 5))
#2 magnitud
ax5[0].plot(f_dig,mag_lin_dig)
ax5[0].plot(f_analog,mag_lin_analog)
ax5[0].set_title("Resp. en frecuencia: Magnitud")
ax5[0].set_xlabel("Frecuencia [Hz]")  
ax5[0].set_ylabel("Magnitud [lineal]")
ax5[0].grid(linestyle='--', which="both")#grilla punteada para escala logaritmica

ax5[1].semilogx(f_dig,mag2_dB)
ax5[1].semilogx(f_analog,mag_dB)
ax5[1].set_title("Resp. en frecuencia: Magnitud")
ax5[1].set_xlabel("Frecuencia [Hz]")  
ax5[1].set_ylabel("Magnitud [dB]")
ax5[1].grid(linestyle='--', which="both")#grilla punteada para escala logaritmica


ax5[2].plot(f_dig,fase_dig)
ax5[2].plot(f_analog,fase_analog)
ax5[2].legend(('Sistema Digital', 'Sistema Analogico'), loc='upper right', shadow=True)
ax5[2].set_title("Fase de la respuesta en frecuencia")
ax5[2].set_xlabel("Frecuencia[rad/s]")
ax5[2].set_ylabel("Angulo[deg]")
ax5[2].grid(linestyle='--', which="both")#grilla punteada

# Transformación Bilineal

## Sistema Analogico

In [11]:
#sistema
w0 = 15*2*np.pi
t_c = np.arange(0,10,0.01)
w = np.arange(0.001,1000*2*np.pi,0.01)

sys_1 = signal.lti([0,w0], [1,w0])

# Respuesta en Frecuencia
w_log, mag_dB, fase = signal.bode(sys_1,w=w)
w, H_analog = signal.freqresp(sys_1,w=w)
mag_lin_analog = abs(H_analog) #magnitud
fase_analog = np.angle(H_analog, deg=True) #/np.pi #fase[rad]

fig6, ax6 = plt.subplots(1, 3, figsize=(16, 5))
#2 magnitud bode
ax6[0].semilogx(w_log/(2*np.pi),mag_dB)
ax6[0].set_title("Diagrama de Bode: Magnitud")
ax6[0].set_xlabel("Frecuencia [Hz]")  
ax6[0].set_ylabel("Magnitud [dB]")
ax6[0].grid(linestyle='--', which="both")#grilla punteada para escala logaritmica

#2 magnitud lineal
ax6[1].plot(w/(2*np.pi),mag_lin_analog)
ax6[1].set_title("Resp. en frecuencia: Magnitud")
ax6[1].set_xlabel("Frecuencia [Hz]")  
ax6[1].set_ylabel("Magnitud [lineal]")
ax6[1].grid(linestyle='--', which="both")#grilla punteada para escala logaritmica

#3 fase
ax6[2].semilogx(w_log,fase)
ax6[2].set_title("Fase de la respuesta en frecuencia")
ax6[2].set_xlabel("Frecuencia [Hz]")
ax6[2].set_ylabel("Angulo[deg]")
ax6[2].grid(linestyle='--', which="both")#grilla punteada
fig6.show()

In [12]:
# Elegimos el tiempo de muestreo acorde
f_max = 1000
print(f_max)
T = 1/(2*f_max)
print(T)

1000
0.0005


### Sistema Digital equivalente

In [13]:
# Obtenemos el sistema digital mediante la utilizacion de la tranformacion bilineal (en la siguiente linea de codigo)
sys_dig_2 = signal.dlti((T*w0/(2+w0*T))*np.array([1,1]), [1,(w0*T-2)/(w0*T+2)])
Omega = np.arange(1/100,np.pi,0.001)

#respuesta en frecuencia
Omega_log, mag2_dB, fase2 = signal.dbode(sys_dig_2, w=Omega)
Omega2, H_dig = signal.dfreqresp(sys_dig_2,w=Omega)
mag_lin_dig = abs(H_dig) #magnitud
fase_dig = np.angle(H_dig, deg = True) #fase[deg]

fig7, ax7 = plt.subplots(1, 3, figsize=(16, 5))
#2 magnitud
ax7[0].plot(Omega/np.pi,mag2_dB)
ax7[0].set_title("Resp. en frecuencia: Magnitud")
ax7[0].set_xlabel("Frecuencia [rad/s]")  
ax7[0].set_ylabel("Magnitud [dB]")
ax7[0].xaxis.set_major_formatter(tck.FormatStrFormatter('%g $\pi$'))
ax7[0].grid(linestyle='--', which="both")#grilla punteada para escala logaritmica

ax7[1].plot(Omega2/np.pi,mag_lin_dig)
ax7[1].set_title("Resp. en frecuencia: Magnitud")
ax7[1].set_xlabel("Frecuencia [rad/s]")  
ax7[1].set_ylabel("Magnitud [lineal]")
ax7[1].xaxis.set_major_formatter(tck.FormatStrFormatter('%g $\pi$'))
ax7[1].grid(linestyle='--', which="both")#grilla punteada para escala logaritmica

#3 fase
ax7[2].plot(Omega/np.pi,fase2)
ax7[2].set_title("Fase de la respuesta en frecuencia")
ax7[2].set_xlabel("Frecuencia[rad/s]")
ax7[2].set_ylabel("Angulo[deg]")
ax7[2].grid(linestyle='--', which="both")#grilla punteada
ax7[2].xaxis.set_major_formatter(tck.FormatStrFormatter('%g $\pi$'))
#ax3.xaxis.set_major_locator(tck.MultipleLocator(base=0.5))
fig7.show()

### Superponemos ambas respuestas en frecuencia

In [14]:
#respuesta en frecuencia
f_dig = Omega2/np.pi*1/(2*T)
f_analog = w/(2*np.pi) 

fig8, ax8 = plt.subplots(1, 3, figsize=(16, 5))
#2 magnitud
ax8[0].plot(f_dig,mag_lin_dig)
ax8[0].plot(f_analog,mag_lin_analog)
ax8[0].set_title("Resp. en frecuencia: Magnitud")
ax8[0].set_xlabel("Frecuencia [Hz]")  
ax8[0].set_ylabel("Magnitud [lineal]")
ax8[0].grid(linestyle='--', which="both")#grilla punteada para escala logaritmica

#ax8[1].semilogx(f_dig,mag2_dB)
#ax8[1].semilogx(f_analog,mag_dB)
ax8[1].plot(f_dig,mag2_dB)
ax8[1].plot(f_analog,mag_dB)
ax8[1].set_title("Resp. en frecuencia: Magnitud")
ax8[1].set_xlabel("Frecuencia [Hz]")  
ax8[1].set_ylabel("Magnitud [dB]")
ax8[1].grid(linestyle='--', which="both")#grilla punteada para escala logaritmica


ax8[2].plot(f_dig,fase_dig)
ax8[2].plot(f_analog,fase_analog)
ax8[2].legend(('Sistema Digital', 'Sistema Analogico'), loc='upper right', shadow=True)
ax8[2].set_title("Fase de la respuesta en frecuencia")
ax8[2].set_xlabel("Frecuencia[rad/s]")
ax8[2].set_ylabel("Angulo[deg]")
ax8[2].grid(linestyle='--', which="both")#grilla punteada

fig8.show()

## Prewarping

In [15]:
#Aplicamos pre-warping sobre la frecuencia de corte e implementamos la transformacion bilineal
w0_prewarp = 2/T*np.tan(w0*T/2)

sys_dig_prewarp = signal.dlti((T*w0_prewarp/(2+w0_prewarp*T))*np.array([1,1]), [1,(w0_prewarp*T-2)/(w0_prewarp*T+2)])
Omega = np.arange(1/100,np.pi,0.001)

#respuesta en frecuencia
Omega_log, mag3_dB, fase3 = signal.dbode(sys_dig_prewarp, w=Omega)
Omega3, H_dig_prewarp = signal.dfreqresp(sys_dig_2,w=Omega)
mag_lin_dig_prewarp = abs(H_dig) #magnitud
fase_dig_prewarp = np.angle(H_dig_prewarp, deg = True) #fase[deg]

f_dig = Omega2/np.pi*1/(2*T)
f_analog = w/(2*np.pi) 

fig9, ax9 = plt.subplots(1, 3, figsize=(16, 5))
#2 magnitud
ax9[0].plot(f_dig,mag_lin_dig_prewarp)
ax9[0].plot(f_dig,mag_lin_dig)
ax9[0].plot(f_analog,mag_lin_analog)
ax9[0].set_title("Resp. en frecuencia: Magnitud")
ax9[0].set_xlabel("Frecuencia [Hz]")  
ax9[0].set_ylabel("Magnitud [lineal]")
ax9[0].grid(linestyle='--', which="both")#grilla punteada para escala logaritmica

#ax9[1].semilogx(f_dig,mag2_dB)
#ax9[1].semilogx(f_analog,mag_dB)
ax9[1].plot(f_dig,mag3_dB)
ax9[1].plot(f_dig,mag2_dB)
ax9[1].plot(f_analog,mag_dB)
#ax9[1].set_xlim(14,17)
#ax9[1].set_ylim(-3.25,-2.75)
ax9[1].set_title("Resp. en frecuencia: Magnitud")
ax9[1].set_xlabel("Frecuencia [Hz]")  
ax9[1].set_ylabel("Magnitud [dB]")
ax9[1].grid(linestyle='--', which="both")#grilla punteada para escala logaritmica


ax9[2].plot(f_dig,fase_dig_prewarp)
ax9[2].plot(f_dig,fase_dig)
ax9[2].plot(f_analog,fase_analog)
ax9[2].legend(('Sistema Digital - Prewarp', 'Sistema Digital','Sistema Analogico'), loc='upper right', shadow=True)
ax9[2].set_title("Fase de la respuesta en frecuencia")
ax9[2].set_xlabel("Frecuencia[rad/s]")
ax9[2].set_ylabel("Angulo[deg]")
ax9[2].grid(linestyle='--', which="both")#grilla punteada

fig9.show()